##### 💡 **라이브러리 불러오기**

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
import torchsummary

---------------------------------------------------------------------------------------

##### 💡 **CIFAR10 데이터 세트 불러오기**

- transforms.Normalize 
    - 특정 평균과 표준편차를 따르는 정규분포를 통해 이미지를 표준화하는 방법
    - CIFAR 10은 3채널 컬러 이미지이므로 각 장의 평균과 표준편차를 정한다. 
    - 첫번째 (0.5, 0.5, 0.5) : 각 채널 당 평균을 할당한 것 (튜플로 입력)
    - 두번째 (0.5, 0.5, 0.5) : 각 채널 당 표준편차를 기입
    - 평균과 표준편차는 학습 전에 가지고 있는 이미지로부터 계산하지만 일단 임의의 값 0.5로 

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


---------------------------------------------------------------------------------------

##### 💡 **GPU 연산 체크하기**

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


---------------------------------------------------------------------------------------

##### 💡 **AlexNet 구축하기**

- AlexNet은 ImageNet 데이터를 위해 만들어졌다.
- ImageNet
    - 1000개의 클래스로 분류되어있는 256x256 또는 224x224 크기를 갖는 이미지
    - 크기가 32x32인 CIFAR10의 이미지는 제대로 동작을 안 할 수 있다.
    - 따라서 **데이터에 맞게 필터의 크기와 보폭 수를 조정해 모델을 구축**

- `self.features` : 합성곱 연산과 풀링 연산이 행해지는 피쳐추출(**Feature extraction**) 부분
    - `nn.Sequential`
        - 순차적으로 행해지는 연산을 한번에 묶을 수 있다. 
        - 괄호 안에는 작성 순서대로 연산이 수행
    - `nn.Conv2d(입력 채널 수, 출력 채널 수, 필터의 크기)`
        - 제일 첫번째 입력 채널 수는 RGB 컬러 이미지의 경우 3(그 후에는 이전 층의 출력값 = 다음 층의 입력값의 크기)
        - 출력 채널 수는 임의의 값
        - `padding` : 해당 층의 입력 피쳐맵의 가장 외곽을 0으로 한 겹 둘러싼다는 의미
        (padding_mode에는 zeros, reflect, replicate, circular)
        - `stride`
    - `nn.MaxPool2d(필터의 크기, 보폭)`


- `self.classifier` : Fully-connected layer(FC)로 구성
    - 처음 들어오는 입력값의 크기 = self.features에서 나온 피쳐맵을 일렬로 편 벡터의 크기
    - CIFAR10은 10개의 클래스를 가진 데이터이므로 마지막 노드 수는 10
    - 나머지 노드 수, 드롭아웃은 임의 지정 가능

In [5]:
class AlexNet(nn.Module) :
    def __init__(self) :
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(3, 64, 3), nn.ReLU(),
                                      nn.MaxPool2d(2, 2),
                                      nn.Conv2d(64, 192, 3, padding=1), nn.ReLU(),
                                      nn.MaxPool2d(2, 2),
                                      nn.Conv2d(192, 384, 3, padding=1), nn.ReLU(),
                                      nn.Conv2d(384, 256, 3, padding=1), nn.ReLU(),
                                      nn.Conv2d(256, 256, 1), nn.ReLU(),
                                      nn.MaxPool2d(2, 2))
                                      
            
        self.classifier = nn.Sequential(nn.Dropout(0.5),
                                        nn.Linear(256*3*3, 1024), nn.ReLU(),
                                        nn.Dropout(0.5),
                                        nn.Linear(1024, 512), nn.ReLU(),
                                        nn.Linear(512, 10))
        
    def forward(self, x) :
        x = self.features(x)
        x = x.view(-1, 256*3*3)
        print(x)
        x = self.classifier(x)
        return x

In [6]:
model = AlexNet().to(device)

In [7]:
torchsummary.summary(model, (3, 32, 32))

tensor([[0.0614, 0.0603, 0.0633,  ..., 0.0712, 0.0528, 0.0554],
        [0.0605, 0.0584, 0.0612,  ..., 0.0676, 0.0531, 0.0583]],
       device='cuda:0', grad_fn=<ViewBackward>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
              ReLU-2           [-1, 64, 30, 30]               0
         MaxPool2d-3           [-1, 64, 15, 15]               0
            Conv2d-4          [-1, 192, 15, 15]         110,784
              ReLU-5          [-1, 192, 15, 15]               0
         MaxPool2d-6            [-1, 192, 7, 7]               0
            Conv2d-7            [-1, 384, 7, 7]         663,936
              ReLU-8            [-1, 384, 7, 7]               0
            Conv2d-9            [-1, 256, 7, 7]         884,992
             ReLU-10            [-1, 256, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]       

/home/ubuntu/anaconda3/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


---------------------------------------------------------------------------------------

##### 💡 **손실 함수 및 최적화 방법 정의하기**

- 다중 분류 문제에서는 Cross Entropy Loss를 기본으로 사용
    - 파이토치에서 제공하는 Cross Entropy Loss는 Softmax 계산까지 포함
    - 모델의 마지막 출력값에 별도의 softmax 적용을 하지 않아도 된다. 
- **GPU 연산을 위해 모델을 불러올 때 .to(device) 반드시 붙여주기**

In [8]:
criterion = nn.CrossEntropyLoss()
alexnet = AlexNet().to(device)
optimizer = optim.Adam(alexnet.parameters(), lr=1e-3)

---------------------------------------------------------------------------------------

##### 💡 **AlexNet 모델 학습하기**

- **GPU 연산을 위해 데이터에도 .to(device) 붙여주기**

In [ ]:
# 그래프를 그리기 위한 loss 저장용 리스트
loss_ = []

# 배치개수
n = len(trainloader)

for epoch in range(50) :
    running_loss = 0.0
    for data in trainloader :
        # 원래는 inputs, labels = data로 하는데 각각 device 붙여주기 위해 인덱스로 따로 뗌
        # 배치 데이터
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        
        # 예측값 산출
        outputs = alexnet(inputs)
        
        # 손실함수 계산
        loss = criterion(outputs, labels)
        
        # 손실 함수 기준으로 역전파 선언
        loss.backward()
        
        # 가중치 최적화
        optimizer.step()
        
        running_loss += loss.item()
        
    loss_.append(running_loss/n)
    print('[%d] loss: %.3f' % (epoch+1, running_loss/len(trainloader)))

---------------------------------------------------------------------------------------

##### 💡 **학습 손실 함수 그래프 그리기**

In [ ]:
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

---------------------------------------------------------------------------------------

##### 💡 **파이토치 모델 저장 및 불러오기**

- 평가가 잘 되었다면 추후 이어서 학습을 하거나 실험 자료를 남기기 위해 모델을 저장해야 한다.

In [14]:
# 모델 저장 경로와 모델평.pth 입력
PATH = './model/cifal_alexnet.pth'

In [16]:
# 모델 저장
torch.save(alexnet.state_dict(), PATH)

FileNotFoundError: [Errno 2] No such file or directory: './model/cifal_alexnet.pth'

In [ ]:
# 모델 저장은 모델을 통째로 저장하는 것이 아니고 모델 파라미터를 저장
# 따라서 저장된 모델을 불러올 때 모델이 선행적으로 선언되어 있어야만 한다.
alexnet = AlexNet().to(device)

# 모델의 파라미터를 불러와 모델에 주입
alexnet.load_state_dict(torch.load(PATH))

---------------------------------------------------------------------------------------

##### 💡 **평가하기**

- `_, predicted = torch.max(outputs, 1)`
    - outputs은 크기가 (배치 크기) x 10인 벡터 형태로 나온다. 
    - 따라서 열 기준으로 가장 큰 원소의 위치가 라벨이 되는 것이기 때문에 최댓값을 열(1) 기주으로 계산하여 예측값을 구한다.
    - `torch.max`는 최댓값, 최댓값의 위치를 산출 (최댓값은 필요 없으므로 _로 처리하여 해당 출력값은 저장 X)
    - 즉, _, predicted는 최댓값의 위치만 predicted에 저장

In [ ]:
# 정답 개수
correct = 0

# 전체 개수
total = 0

# 평가 시에는 requires_grad 비활성화
with torch.no_grad() :
    # 평가 시에는 드롭아웃 등과 같은 정규화 작업 X (비활성화)
    alexnet.eval()
    for data in testloader :
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Test accuracy: %.2f %%' %(100*correct/total))